In [1]:
import re
import folium
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Airbnb listings
January 20 with license data

In [3]:
airbnb = pd.read_csv('../../data/insideairbnb/boston_feb20.csv', low_memory=0)
#df.index = pd.DatetimeIndex(df['last_scraped'])
#airbnb = airbnb[['last_scraped', 'requires_license', 'license', 'jurisdiction_names', 'price',
                 #'latitude', 'longitude', 'is_location_exact', 'street', 'zipcode']]
airbnb.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,3781,https://www.airbnb.com/rooms/3781,20200213183241,2020-02-13,HARBORSIDE-Walk to subway,Fully separate apartment in a two apartment bu...,This is a totally separate apartment located o...,Fully separate apartment in a two apartment bu...,none,"Mostly quiet ( no loud music, no crowed sidewa...",...,f,f,super_strict_30,f,f,1,1,0,0,0.29
1,5506,https://www.airbnb.com/rooms/5506,20200213183241,2020-02-13,**$79 Special ** Private! Minutes to center!,"Private guest room with private bath, You do n...",**THE BEST Value in BOSTON!!*** PRIVATE GUEST ...,"Private guest room with private bath, You do n...",none,"Peacful, Architecturally interesting, historic...",...,t,f,strict_14_with_grace_period,f,f,6,6,0,0,0.80
2,6695,https://www.airbnb.com/rooms/6695,20200213183241,2020-02-13,$99 Special!! Home Away! Condo,"Comfortable, Fully Equipped private apartment...",** WELCOME *** FULL PRIVATE APARTMENT In a His...,"Comfortable, Fully Equipped private apartment...",none,"Peaceful, Architecturally interesting, histori...",...,t,f,strict_14_with_grace_period,f,f,6,6,0,0,0.90
3,8789,https://www.airbnb.com/rooms/8789,20200213183241,2020-02-13,Curved Glass Studio/1bd facing Park,"Bright, 1 bed with curved glass windows facing...",Fully Furnished studio with enclosed bedroom. ...,"Bright, 1 bed with curved glass windows facing...",none,Beacon Hill is a historic neighborhood filled ...,...,f,f,strict_14_with_grace_period,f,f,8,8,0,0,0.36
4,10730,https://www.airbnb.com/rooms/10730,20200213183241,2020-02-13,Bright 1bed facing Golden Dome,"Bright, spacious unit, new galley kitchen, new...",Bright one bed facing the golden dome of the S...,"Bright, spacious unit, new galley kitchen, new...",none,Beacon Hill is located downtown and is conveni...,...,f,f,strict_14_with_grace_period,f,f,8,8,0,0,0.24


In [5]:
na_license = airbnb[(airbnb['requires_license']=='t')&(airbnb['license'].isna())]
na_license

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,3781,https://www.airbnb.com/rooms/3781,20200213183241,2020-02-13,HARBORSIDE-Walk to subway,Fully separate apartment in a two apartment bu...,This is a totally separate apartment located o...,Fully separate apartment in a two apartment bu...,none,"Mostly quiet ( no loud music, no crowed sidewa...",...,f,f,super_strict_30,f,f,1,1,0,0,0.29
3,8789,https://www.airbnb.com/rooms/8789,20200213183241,2020-02-13,Curved Glass Studio/1bd facing Park,"Bright, 1 bed with curved glass windows facing...",Fully Furnished studio with enclosed bedroom. ...,"Bright, 1 bed with curved glass windows facing...",none,Beacon Hill is a historic neighborhood filled ...,...,f,f,strict_14_with_grace_period,f,f,8,8,0,0,0.36
4,10730,https://www.airbnb.com/rooms/10730,20200213183241,2020-02-13,Bright 1bed facing Golden Dome,"Bright, spacious unit, new galley kitchen, new...",Bright one bed facing the golden dome of the S...,"Bright, spacious unit, new galley kitchen, new...",none,Beacon Hill is located downtown and is conveni...,...,f,f,strict_14_with_grace_period,f,f,8,8,0,0,0.24
5,10811,https://www.airbnb.com/rooms/10811,20200213183241,2020-02-13,"Back Bay Apt Studio-3 blocks to Pru center & ""T""",Stunning Back Bay furnished studio apartment. ...,"Back Bay Studio Apt - Private bath, A/C, Cabl...",Stunning Back Bay furnished studio apartment. ...,none,A one-square mile neighborhood that is arguabl...,...,f,f,strict_14_with_grace_period,f,f,9,9,0,0,0.18
6,10813,https://www.airbnb.com/rooms/10813,20200213183241,2020-02-13,"Back Bay Apt-blocks to subway, Newbury St, The...",Stunning Back Bay furnished studio apartment. ...,"Back Bay Studio Apt - Private bath, A/C, Cabl...",Stunning Back Bay furnished studio apartment. ...,none,Wander around this quintessential neighborhood...,...,f,f,strict_14_with_grace_period,f,f,9,9,0,0,2.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3887,42092767,https://www.airbnb.com/rooms/42092767,20200213183241,2020-02-13,Luxury 2br 2ba apt corner unit with view,Great luxury 2br 2ba apt in Boston has every t...,NaN,Great luxury 2br 2ba apt in Boston has every t...,none,NaN,...,t,f,flexible,f,f,10,10,0,0,NaN
3889,42114365,https://www.airbnb.com/rooms/42114365,20200213183241,2020-02-13,Luxury 2br with charles river view,"Be close to everything, downtown, mall, Massac...",NaN,"Be close to everything, downtown, mall, Massac...",none,NaN,...,t,f,flexible,f,f,10,10,0,0,NaN
3895,42197186,https://www.airbnb.com/rooms/42197186,20200213183241,2020-02-13,Private room in a chill Southie condo,"Clean, modern, and fully furnished 2 bedroom/ ...",Amenities included in price are: - in-unit was...,"Clean, modern, and fully furnished 2 bedroom/ ...",none,"Southie isn't too diverse, but downtown, seapo...",...,t,f,flexible,f,f,1,0,1,0,NaN
3897,42208511,https://www.airbnb.com/rooms/42208511,20200213183241,2020-02-13,Luxury 2br 2ba close to Boston's hospital,"Live here close to every thing, here you are t...",NaN,"Live here close to every thing, here you are t...",none,NaN,...,t,f,flexible,f,f,10,10,0,0,NaN


In [203]:
na_license_coords = zip(na_license['license'],
                             zip(na_license['latitude'],
                                 na_license['longitude']))

notna_license = airbnb_fenway[(airbnb_fenway['requires_license']=='t')&(airbnb_fenway['license'].notna())]
notna_license_coords = zip(notna_license['license'],
                                zip(notna_license['latitude'],
                                    notna_license['longitude']))

In [204]:
m = folium.Map(location=[42.342943, -71.099371],
               tiles='Stamen Toner',
               zoom_start=14)

for license, coord in na_license_coords:
    folium.Circle(
        radius=20,
        location=coord,
        popup=license,
        color='orange',
        fill=True,
    ).add_to(m)

for license, coord in notna_license_coords:
    folium.Circle(
        radius=20,
        location=coord,
        popup=license,
        color='lightblue',
        fill=True,
    ).add_to(m)
    
m

---

## City of Boston Data
### 1. Map SAM coords onto STRE data

In [217]:
samref = pd.read_csv('../data/cityofboston/sam_reference.csv', low_memory=0)
samref = samref[['X', 'Y', 'SAM_ADDRESS_ID']].rename(
    columns={'Y': 'latitude', 'X':'longitude', 'SAM_ADDRESS_ID':'SAM'}).set_index('SAM')

stre = pd.read_csv('../data/cityofboston/stre-feb-4.csv', low_memory=0)
stre = stre[['sam_address_id', 'issued_registration', 'home-share eligible',
             'violations in the last 6 months', 'unit owner-occupied']].rename(
    columns={'sam_address_id':'SAM'}).set_index('SAM')

stre = stre.join(samref, how='inner')
stre.head()

,issued_registration,home-share eligible,violations in the last 6 months,unit owner-occupied,longitude,latitude
SAM,,,,,,
344711,N,N,0,N,-71.099847,42.318650
355607,N,N,0,N,-71.077679,42.347270
421113,N,N,0,N,-71.062141,42.354299
139026,N,Y,0,Y,-71.106291,42.309459
177284,N,N,0,N,-71.086291,42.326679


In [218]:
# InsideAirbnb:
na_license = airbnb_fenway[(airbnb_fenway['requires_license']=='t')&(airbnb_fenway['license'].isna())]
na_license_coords = zip(na_license['license'],
                             zip(na_license['latitude'],
                                 na_license['longitude']))

notna_license = airbnb_fenway[(airbnb_fenway['requires_license']=='t')&(airbnb_fenway['license'].notna())]
notna_license_coords = zip(notna_license['license'],
                                zip(notna_license['latitude'],
                                    notna_license['longitude']))

# City of Boston:
not_registered = stre_fenway[(stre_fenway['issued_registration']=='N') &
                             (stre_fenway['violations in the last 6 months']>4)] # > 5 violations in past 6 mos
not_registered_coords = zip(not_registered['violations in the last 6 months'],
                             zip(not_registered['latitude'],
                                 not_registered['longitude']))

is_registered = stre_fenway[(stre_fenway['issued_registration']=='Y')]
is_registered_coords = zip(is_registered['violations in the last 6 months'],
                             zip(is_registered['latitude'],
                                 is_registered['longitude']))

In [219]:
m = folium.Map(location=[42.342943, -71.099371],
               tiles='Stamen Toner',
               zoom_start=14)



# InsideAirbnb data:

for license, coord in notna_license_coords:  # exempted or licensed: PINK (loc. approx.)
    folium.Circle(
        radius=10,
        location=coord,
        popup=license,
        color='pink',
        fill=True,
    ).add_to(m)
for license, coord in na_license_coords: , # NaN license: RED (loc. approx.)
    folium.Circle(
        radius=10,
        location=coord,
        popup=license,
        color='crimson'
        fill=True,
    ).add_to(m)



# City of Boston STRE data:

for vios, coord in is_registered_coords: # is registered = BLUE (loc. exact)
    folium.Circle(
        radius=20,
        location=coord,
        popup=vios,
        color='blue',
        fill=True,
    ).add_to(m)
for vios, coord in not_registered_coords:  # >=5 vios. past 6 mos. = ORANGE (loc. exact)
    folium.Circle(
        radius=20,
        location=coord,
        popup=vios,
        color='orange',
        fill=True,
    ).add_to(m)
    
m

In [ ]:
stre['issued_registration'].value_counts()

In [220]:
# InsideAirbnb:
na_license = airbnb_fenway[(airbnb_fenway['requires_license']=='t')&(airbnb_fenway['license'].isna())]
na_license_coords = zip(na_license['license'],
                             zip(na_license['latitude'],
                                 na_license['longitude']))

notna_license = airbnb_fenway[(airbnb_fenway['requires_license']=='t')&(airbnb_fenway['license'].notna())]
notna_license_coords = zip(notna_license['license'],
                                zip(notna_license['latitude'],
                                    notna_license['longitude']))

# City of Boston:
not_registered = stre_fenway[(stre_fenway['issued_registration']=='N') &
                             (stre_fenway['violations in the last 6 months']>4)] # > 5 violations in past 6 mos
not_registered_coords = zip(not_registered['violations in the last 6 months'],
                             zip(not_registered['latitude'],
                                 not_registered['longitude']))

is_registered = stre_fenway[(stre_fenway['issued_registration']=='Y')]
is_registered_coords = zip(is_registered['violations in the last 6 months'],
                             zip(is_registered['latitude'],
                                 is_registered['longitude']))

In [221]:
print('——— Fenway | January 2020 ———')
print('(42.352737:42.332079||71.081596:-71.112581)')
print(f'registrations issued: {len(is_registered)}') # feb 4 stre
print(f'airbnb listings: {len(airbnb_fenway)}')

——— Fenway | January 2020 ———
(42.352737:42.332079||71.081596:-71.112581)
registrations issued: 30
airbnb listings: 396


In [210]:
print('——— Fenway | January 2020 ———')
print('(42.352737:42.332079||71.081596:-71.112581)')
print(f'registrations issued: {len(is_registered)}') # feb 22 stre
print(f'airbnb listings: {len(airbnb_fenway)}')

——— Fenway | January 2020 ———
(42.352737:42.332079||71.081596:-71.112581)
registrations issued: 30
airbnb listings: 396


Exempt: This listing is a unit that has contracts with hospitals                     285
Exempt: This listing is a unit used for furnished institutional or business stays    138
Exempt: This listing is a hotel or motel                                              73
Exempt: You host a unit used for furnished institutional or business stays            72
STR-918313                                                                            47
STR-390690                                                                            39
Exempt: You host a hotel or motel                                                     36
STR-988180                                                                            24
Exempt: You host a unit that has contracts with hospitals                             23
STR-974333                                                                            21
LB-101723                                                                             21
STR-986025           

In [218]:
# InsideAirbnb:
na_license = airbnb_fenway[(airbnb_fenway['requires_license']=='t')&(airbnb_fenway['license'].isna())]
na_license_coords = zip(na_license['license'],
                             zip(na_license['latitude'],
                                 na_license['longitude']))

notna_license = airbnb_fenway[(airbnb_fenway['requires_license']=='t')&(airbnb_fenway['license'].notna())]
notna_license_coords = zip(notna_license['license'],
                                zip(notna_license['latitude'],
                                    notna_license['longitude']))

# City of Boston:
not_registered = stre_fenway[(stre_fenway['issued_registration']=='N') &
                             (stre_fenway['violations in the last 6 months']>4)] # > 5 violations in past 6 mos
not_registered_coords = zip(not_registered['violations in the last 6 months'],
                             zip(not_registered['latitude'],
                                 not_registered['longitude']))

is_registered = stre_fenway[(stre_fenway['issued_registration']=='Y')]
is_registered_coords = zip(is_registered['violations in the last 6 months'],
                             zip(is_registered['latitude'],
                                 is_registered['longitude']))